# Peer reviewed assignment Capstone week 3
### Pandas Dataframe creation
Scraping the given Wikipedia page to obtain the data and to transform it to create it a pandas Dataframe

In [82]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [83]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = BeautifulSoup(html, 'html.parser')
table = content.find('table')

Iterating through all the rows in order to save the contents in the variable "d". After this, using the variable "d" to create the pandas dataframe called "data_frame". Then displaying the first 5 rows of "data_frame" for viewing the data.

In [86]:
d = []
for tr in table.find_all('tr')[1:]:
    d_rows = tr.find_all('td')
    d.append([cell.text for cell in d_rows])
data_frame = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
data_frame.head(5)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


## Data Cleaning
Removing the rows where Borough has a 'Not assigned' value

In [88]:
data_frame = data_frame[data_frame['Borough'] != 'Not assigned\n']
data_frame.reset_index(drop = True, inplace = True)
data_frame.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


Now assigning "Neighbourhood" value to "Borough" value, if it has a "Not assigned". Also notice, the data in all columns has a "\n" character at the end, so getting rid of this character as well.

In [89]:
data_frame['Neighborhood'] = data_frame['Neighborhood'].str.split('\n', expand = True)
data_frame['Borough'] = data_frame['Borough'].str.split('\n', expand = True)
data_frame['PostalCode'] = data_frame['PostalCode'].str.split('\n', expand = True)
data_frame.loc[data_frame['Neighborhood'] == 'Not assigned', 'Neighborhood'] = data_frame.loc[data_frame['Neighborhood'] == 'Not assigned', 'Borough']

Finally, combining the neighborhoods which have the same postal code. These neighborhoods will be separated with a comma.

In [92]:
data_frame = data_frame.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
# viewing the first 5 rows
data_frame.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Saving the cleaned dataframe as a CSV file and then printing the number of rows in the dataframe

In [94]:
data_frame.to_csv('Dataframe.csv', index = False)

In [95]:
print("Shape : {}".format(data_frame.shape))

Shape : (103, 3)
